### Imports

In [1]:
import datetime
import numpy as np
import pandas as pd
import re
import crawler as cr

### Main Logic

In [2]:
# General configuration steps
# What cities do you want to search? Just the ones in your state? Or in your region?
# Modifying state_list will automatically modify city_list
states = cr.get_craigslist_urls_by_state()
state_list = ['texas', 'arkansas', 'alabama', 'mississippi', 'new mexico', 'colorado', 'oklahoma', 'louisiana', 'kansas']
city_list = []
for state in state_list:
    city_list += states[state]
print(city_list)

# Search strings and stop strings will filter your results
# If an item's title does not contain any of the search strings, then that item will be dropped
# If an item's title contains any of the stop strings, then that item will be dropped
search_strings = ['2009', '2010', '2011', '2012', '2013', '2014', '2015']
stop_strings = ['gti', 'chevrolet', 'chevy', 'ford', '3.0', 'touareg', 'nissan', 'gmc', 'chrysler',
                'dodge', 'saturn', 'mercedes', 'honda', 'buick', 'toyota', 'jeep', 'lincoln',
                'scion', 'yamaha', ]


['abilene', 'amarillo', 'austin', 'beaumont', 'brownsville', 'collegestation', 'corpuschristi', 'dallas', 'nacogdoches', 'delrio', 'elpaso', 'galveston', 'houston', 'killeen', 'laredo', 'lubbock', 'mcallen', 'odessa', 'sanangelo', 'sanantonio', 'sanmarcos', 'bigbend', 'texoma', 'easttexas', 'victoriatx', 'waco', 'wichitafalls', 'fayar', 'fortsmith', 'jonesboro', 'littlerock', 'texarkana', 'auburn', 'bham', 'dothan', 'shoals', 'gadsden', 'huntsville', 'mobile', 'montgomery', 'tuscaloosa', 'gulfport', 'hattiesburg', 'jackson', 'meridian', 'northmiss', 'natchez', 'albuquerque', 'clovis', 'farmington', 'lascruces', 'roswell', 'santafe', 'boulder', 'cosprings', 'denver', 'eastco', 'fortcollins', 'rockies', 'pueblo', 'westslope', 'lawton', 'enid', 'oklahomacity', 'stillwater', 'tulsa', 'batonrouge', 'cenla', 'houma', 'lafayette', 'lakecharles', 'monroe', 'neworleans', 'shreveport', 'lawrence', 'ksu', 'nwks', 'salina', 'seks', 'swks', 'topeka', 'wichita']


In [7]:
# This is where we run the meat of the program

ds = [] # List of dictionaries of data
city_list = ['dallas'] # Make city_list short for testing

# TODO:
# cshaley: 2016/10/17
# Restructure this to make it more linear.
# i.e. Create a dataframe with all of the links to load for all of the cities,
#      and then load all of those links.
# Drop duplicate entries before loading them?

# TODO
# cshaley: 2016/10/18
# Restructure the whole program - put in .py files and make it object oriented?

for city in city_list:
    # Create the query URL
    url = "https://{}.craigslist.org/search/cta?query=tdi".format(city)
    
    # Get links to all of the items in the search results
    tlst, hlst = cr.get_sale_item_links(url)
    
    # If there were any results
    if tlst and hlst:
        # Create dataframe with craigslist items for sale as rows
        df = pd.DataFrame(list(zip(tlst, hlst)), columns=['Title', 'Link'])
        
        # Filter before loading individual URLS as loading each URL is slow.
        # Filter out items that don't contain any search strings
        df = df[[True \
                 if any(s in str(r, 'utf-8').lower() \
                        for s in search_strings) \
                 else False \
                 for r in df['Title']]]

        # Filter out items that contain stop strings
        df = df[[False \
                 if any(s in str(r, 'utf-8').lower() \
                        for s in stop_strings) \
                 else True \
                 for r in df['Title']]]
        
        # Load each link and get the item attributes for each link.
        # Create a dictionary and append it t
        for lnk, title in zip(df['Link'].values, df['Title'].values):
            d = cr.get_sale_item_attrs(lnk)
            d['Link'] = lnk
            d['Title'] = title
            ds.append(d)

# Create a dataframe from the list of dictionaries
car_data = pd.DataFrame(ds)

### Viewing and Saving

In [10]:
# Print Number of results and clean data
print("Raw car data number of results: {}".format(len(car_data.index)))
cleaned_car_data = car_data.drop_duplicates()
print("Cleaned car data number of results: {}".format(len(cleaned_car_data.index)))
cleaned_car_data = cleaned_car_data[cleaned_car_data[u'title status: ']=='clean']
print("Cleaned car data with clean title number of results: {}".format(len(cleaned_car_data.index)))

Raw car data number of results: 13
Cleaned car data number of results: 13
Cleaned car data with clean title number of results: 13


In [11]:
car_data.head()

,Description,Link,Price,Title,VIN:,condition:,cylinders:,drive:,fuel:,odometer:,paint color:,size:,title status:,transmission:,type:
0,2011 *Audi* *Q7* *TDI* *QUATTRO* *PRESTIGE* NA...,http://dallas.craigslist.org/dal/ctd/583334665...,$26995,b'2011 *Audi* *Q7* *TDI* *QUATTRO* *PRESTIGE* ...,WA1VMAFE0BD008107,NaN,NaN,NaN,gas,93358,NaN,NaN,clean,automatic,NaN
1,Black / Tan 2009 Volkswagen Jetta Sedan DSG TD...,http://dallas.craigslist.org/mdf/cto/581621517...,$8495,b'2009 Volkswagen Jetta Sedan DSG TDI',NaN,NaN,NaN,NaN,diesel,107000,NaN,NaN,clean,automatic,NaN
2,Selling a black Audi Q7 TDI Quattro Premium Pl...,http://dallas.craigslist.org/dal/cto/583608168...,$40000,b'2014 Audi Q7 TDI',WA1LMAFE0ED005011,like new,6 cylinders,4wd,diesel,30251,black,NaN,clean,automatic,SUV
3,2014 Volkswagen Jetta Sedan diesel 500.00 tota...,http://dallas.craigslist.org/mdf/ctd/582852257...,$19995,b'2014 Volkswagen Jetta Sedan 4dr DSG TDI 500....,3VWLL7AJ7EM237221,excellent,4 cylinders,fwd,diesel,66889,silver,compact,clean,automatic,sedan
4,2012 *Volkswagen* *Jetta* *Sedan* TDI; $795 Do...,http://dallas.craigslist.org/dal/ctd/579702982...,$10995,b'2012 *Volkswagen* *Jetta* *Sedan* TDI; $795 ...,3VWLL7AJ9CM332327,NaN,NaN,NaN,other,81289,NaN,NaN,clean,automatic,NaN


In [ ]:
cleaned_car_data.head()

In [ ]:
# Write data to disk - with a timestamp so you dont overwrite anything on accident
now = re.sub('[ .:-]', '', str(datetime.datetime.now()))
car_data.to_csv('raw_car_data_{}.csv'.format(now), index=False, encoding='utf-8')
cleaned_car_data.to_csv('cleaned_car_data_{}.csv'.format(now), index=False, encoding='utf-8')

In [ ]:
# Read it for a sanity check
read_car_data = pd.read_csv('raw_car_data_{}.csv'.format(now))

In [ ]:
read_car_data.head()